In [41]:
import pandas as pd
import spacy
import numpy as np
import pickle
import spacy
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
names=["large","mid","small"]

In [42]:
embeddings_dictionary = dict()
glove_file = open('./glove.6B.100d.txt', encoding="utf8")
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [49]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [24]:
df=pd.read_csv('./news_data.csv')

In [35]:
df.shape

(2438, 4)

In [28]:
corpus_dates = list(df['date'])
corpus = list(df['data'])

In [91]:
len(corpus_dates)

1198

In [29]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [31]:
vocab_length = len(word_tokenizer.word_index) + 1

In [33]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
# print(embedded_sentences)

In [50]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

In [43]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [118]:
length_long_sentence

28

In [59]:
len(padded_sentences)

2438

In [113]:
len(embedding_matrix[0])

100

In [114]:
with open("embedding_matrix.pkl","wb") as f:
        pickle.dump(embedding_matrix,f)

In [115]:
with open("./embedding_matrix.pkl","rb")as f:
    matrix=pickle.load(f)

In [117]:
# matrix

In [61]:
data = {}
for i,date in enumerate(corpus_dates):
    if date in data.keys():
        data[date].append(padded_sentences[i])
    else:
        data[date] = [padded_sentences[i]]

In [92]:
data_vec={}
# count=0
for date in data:
    temp=[]
    for text in data[date]:
#         doc=model(text)
        temp.append(text)
#map the vector mean to that day
    if len(temp) > 1:
        meank=np.mean(np.array(temp,dtype=np.float32),axis=0)
    else:
        meank = temp[0]
    data_vec[date]=meank

In [95]:
data_vec['2011-01-01']

array([2391,  100, 2392,    1, 2393,  897, 1127, 2394,  725,   58,   74,
          6,    7,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
# data_vec
with open("small-new-vec.pkl","wb") as f:
        pickle.dump(data_vec,f)

# Converting the historical prices

In [120]:
sensex_data = pd.read_csv('BSESENSEX.csv',parse_dates=True)

# Pre-processing

In [68]:
import pandas as pd
import spacy
import numpy as np
import pickle
import spacy
import numpy as np
# names=["large","mid","small"]
vectors={"large":None,"mid":None,"small":None}
final_data={"large":None,"mid":None,"small":None}
pricename={'large':'','mid':'mid','small':'small'}

In [97]:
with open("small-new-vec.pkl","rb")as f:
    vectors['small']=pickle.load(f)

In [98]:
len(vectors['small'])

1198

In [99]:
vectors['small']['2011-01-01']

array([2391,  100, 2392,    1, 2393,  897, 1127, 2394,  725,   58,   74,
          6,    7,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [103]:
final_data={}
# df=pd.read_csv('./'+name+'-final.csv')
df2=pd.read_csv("./nifty"+pricename[name]+"--50.csv")
price=df2.sort_values(["date"])
sortdf=df.sort_values(['date'])

change_price=[]
# print("Setting up the change in price for "+name)
for i in range(len(price)-1):
    change_price.append(1*(float(price.iloc[i+1]['Change-close'])>0))

In [101]:
st=[]
mt=[]
lt=[]
mapping={"long":lt,"medium":mt,"short":st}
uniq_dt=price['date'].unique()
for i in range(len(uniq_dt)):
    #last but one since the next day is always for prediction.
    if(i<len(uniq_dt)-1):
        st.append(uniq_dt[i])
        if(i-7>=0):
            mt.append(uniq_dt[i-7:i])
        else:
            mt.append(uniq_dt[:i+1])
        if(i-30>=0):
            lt.append(uniq_dt[i-30:i])
        else:
            lt.append(uniq_dt[:i+1])
# print("Doing for "+name)

In [77]:
# mt

In [105]:
#here term is for the time period, long, medium and short!
for term in mapping:
    count=0
    for dat in mapping[term]:
        temp=[]
        if(type(dat)!=str):
            for entry in dat:
                try:
                    temp.append(vectors[name][entry])
                except Exception as e:
                    pass
            if(len(temp)!=0):
                if term in final_data.keys():
                    final_data[term].append(np.array(temp))
                else:
                    final_data[term] = [np.array(temp)]
            else:
                if term in final_data.keys():
                    final_data[term].append(np.zeros(28))
                else:
                    final_data[term] = [np.zeros(28)]
#                 final_data[term].append(np.zeros(28))
        else:
            try:
                if term in final_data.keys():
                    final_data[term].append(vectors[name][dat])
                else:
                    final_data[term] = [vectors[name][dat]]
#                 final_data[term].append(vectors[name][dat])
            except Exception as e:
                if term in final_data.keys():
                    final_data[term].append(np.zeros(28))
                else:
                    final_data[term] = [np.zeros(28)]
#                 final_data[term].append(np.zeros(28))
        count+=1

In [108]:
for term in ['long','medium','short']:
    if(term=='long'):
        wall=30
    elif(term=='medium'):
        wall=7
    else:
        wall=1
    print("Resizing all vectors to same size for "  +name+" "+term)
    for dat in range(len(final_data[term])):
        ref=len(final_data[term][dat])
        if(ref==28):
            final_data[term][dat]=np.zeros((wall,28))
            continue
        else:
            if(wall-ref>0):
                final_data[term][dat]=np.vstack((final_data[term][dat],np.zeros((wall-ref,28))))
print("Shapes after resize")
for term in final_data:
    final_data[term]=np.array(final_data[term])
    print(" for "+term+"-"+str(final_data[term].shape))
print("Length of the change in price",len(change_price))
dt={"X":final_data,"Y":np.array(change_price)}
with open("small-new-lms_vec_training.pkl","wb")as f:
    pickle.dump(dt,f)
print("done for "+name)
print("#-"*15)
print("Pre processing done !")

Resizing all vectors to same size for small long
Resizing all vectors to same size for small medium
Resizing all vectors to same size for small short
Shapes after resize
 for long-(1672, 30, 28)
 for medium-(1672, 7, 28)
 for short-(1672, 1, 28)
Length of the change in price 1672
done for small
#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
Pre processing done !
